In [ ]:
!pip install simpletransformers

In [ ]:
import logging
import pandas as pd
import wandb
import torch
from simpletransformers.classification import MultiLabelClassificationModel
from simpletransformers.config.model_args import MultiLabelClassificationArgs


In [ ]:
def process_labels(train):
    train['labels'] = ''
    for index, row in train.iterrows():
       train['labels'][index] = [row['misogynous'], row['shaming'], row['stereotype'], row['objectification'], row['violence']]

    train_data = train[['Text Transcription','labels']]
    train_data = train_data.rename(columns={'Text Transcription': 'text'})
    print(train_data.head(5))
    return train_data

In [ ]:
train = pd.read_csv('../input/mamidata/training/TRAINING/training.csv', sep='\t')
train_data = process_labels(train)
print(train_data.head(5))

In [ ]:
model_args = MultiLabelClassificationArgs(num_train_epochs=1)
model_args.num_train_epochs = 20
model_args.threshold = 0.8
model_args.no_save = True
model_args.overwrite_output_dir=True

cuda_available = torch.cuda.is_available()

model = MultiLabelClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=5,
    args=model_args,
    use_cuda=cuda_available
)

In [ ]:
# Train the model
model.train_model(train_data,
                      show_running_loss=True,
                      use_early_stopping=True
                 )

In [ ]:
# Get the test list
# print('--------')
test = pd.read_csv('../input/mamidata/MAMI_test/MAMA_test/Test.csv', sep='\t')
test_original = test
test = test.rename(columns={'Text Transcription': 'text'})
test = test['text']
print("test.head(5): ", test.head(5))

In [ ]:
predictions, raw_outputs = model.predict(list(test))

print(predictions)

In [ ]:
with open("answer_0130_multi_label_taskB.txt", 'w', encoding="utf-8") as f:
    for filename, label in zip(test_original["file_name"], predictions):
        print(filename, label)
        f.write(f"{filename}\t{label}\n")